Find and draw largest contour:  
https://stackoverflow.com/questions/44588279/find-and-draw-the-largest-contour-in-opencv-on-a-specific-color-python  

Straighten a crooked image:  
https://stackoverflow.com/questions/41995916/opencv-straighten-an-image-with-python  

Find all bounding boxes:  
https://stackoverflow.com/questions/21104664/extract-all-bounding-boxes-using-opencv-python  

Deskew images (This one has been the most helpful, the others...no):  
https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/  


In [1]:
from PIL import Image
from pathlib import Path
from itertools import tee
from os import path
import cv2
import numpy as np

rotated = "./rotated"

test = "./rotated/small.jpg"

In [2]:
def load(img):
    # load the image from disk
    image = cv2.imread(str(img))

    # convert to grayscale, flip foreground and background
    # foreground is now "white" and the background is "black"
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)

    # threshold the image, setting all foreground pixels to 255 and all background pixels to 0
#     thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    return image, cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [3]:
def rotate(image, angle):
    # rotate the image to deskew it
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [7]:
# collect image paths
gen, image_paths = tee(Path(rotated).iterdir())
for img_path in image_paths:
    if img_path.suffix == ".jpg":
        image, thresh = load(img_path)
        
        # grab the (x, y) coordinates of all pixel values that
        # are greater than zero, then use these coordinates to
        # compute a rotated bounding box that contains all
        # coordinates
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]

        # the `cv2.minAreaRect` function returns values in the
        # range [-90, 0); as the rectangle rotates clockwise the
        # returned angle trends to 0 -- in this special case we
        # need to add 90 degrees to the angle
        if angle < -45:
            angle = -(90 + angle)
        # otherwise, just take the inverse of the angle to make
        # it positive
        else:
            angle = -angle
        
#         rotate(image, angle)

        # exaggerate the angle to set a cutoff point
#         score = (angle*100)*(angle*100)
#         print(f"{str(img_path)}, angle: {angle}, score: {score}")
        
        
        # draw the correction angle on the image so we can validate it
        # cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        # show the output image
        # print("[INFO] angle: {:.3f}".format(angle))
        # cv2.imshow("Input", image)
        # cv2.imshow("Rotated", rotated)
        # cv2.waitKey(0)

rotated/alphabet00794.jpg, angle: 0.7862328886985779, score: 6181.6215527131035
rotated/alphabet00793.jpg, angle: 0.23036132752895355, score: 530.6634122090181
rotated/alphabet00787.jpg, angle: 0.3230677843093872, score: 1043.7279325857673
rotated/alphabet00368.jpg, angle: 0.8541030883789062, score: 7294.920855783857
rotated/alphabet00369.jpg, angle: 0.8538321852684021, score: 7290.294006002149
rotated/alphabet00786.jpg, angle: -0.118072509765625, score: 139.4111756235361
rotated/alphabet00792.jpg, angle: 0.31421688199043274, score: 987.3224892778953
rotated/alphabet00009.jpg, angle: 0.050132401287555695, score: 25.132576588565158
rotated/alphabet00790.jpg, angle: 0.31873056292533875, score: 1015.8917174270332
rotated/alphabet00791.jpg, angle: 0.31863072514533997, score: 1015.2553900664518
rotated/alphabet00785.jpg, angle: 0.15057988464832306, score: 226.74301660702278
rotated/alphabet00008.jpg, angle: 0.029352344572544098, score: 8.615601319053589
rotated/alphabet00005.jpg, angle: -0.